In [1]:
import os
import pandas as pd
from data_loader import label_str2int_dict

rationale_names = ['td','cs']

print(os.getcwd())
os.chdir('/home/lyq/DataSet/FakeNews/gossipcop')
print(os.getcwd())

/home/lyq/PycharmProjects/QwenVLRationaleGenerate/data/gossipcop
/home/lyq/DataSet/FakeNews/gossipcop


In [2]:
df = pd.read_csv('gossipcop.csv')

df = df.rename(columns={
    'id':"source_id",
    'text':'content',
})

df['label'] = df['label'].apply(lambda x : label_str2int_dict[x])


df.head(10)

,source_id,content,label,publish_date
0,gossipcop-2493749932,Congratulations might be in order for Miley Cy...,1,1.529680e+09
1,gossipcop-941805037,Thousands are taking the streets to protest Pr...,1,1.492292e+09
2,gossipcop-2547891536,We'd venture to say that Cindy Crawford's daug...,1,1.458333e+09
3,gossipcop-5189580095,"During the summer of 1995, John F. Kennedy Jr....",1,1.498769e+09
4,gossipcop-9588339534,"Many stars saw great success in 2016, but the ...",1,1.481818e+09
5,gossipcop-8753274298,Caitlyn Jenner is trying to keep her romantic ...,1,1.533664e+09
6,gossipcop-9878194459,So while the internet is busy obsessing over J...,1,1.531127e+09
7,gossipcop-9521617242,Next stop on the 2018 award season train? The ...,1,1.515726e+09
8,gossipcop-2210288941,"Angelina Jolie, 43, “really hasn’t come to ter...",1,1.534090e+09
9,gossipcop-3783917182,We should not be content to pass health care l...,1,1.506010e+09


In [3]:


for rationale_name in rationale_names:
    rationale_df = pd.read_csv(f'{rationale_name}.csv')
    rationale_df = rationale_df.dropna(subset=['authenticity','reason'])
    if rationale_df.shape[0] != df.shape[0]:
        raise Exception(f'{rationale_name} len : {rationale_df.shape[0]} != {df.shape[0]}')
    df = df.merge(rationale_df,on='source_id',how='left')
    df = df.rename(columns={
        'authenticity':f'{rationale_name}_pred',
        'reason': f'{rationale_name}_rationale',
    })
    df[f'{rationale_name}_pred'] = df[f'{rationale_name}_pred'].apply(lambda x: label_str2int_dict[x])
    print(df.columns)
    df[f'{rationale_name}_acc'] = df.apply(lambda x : int(x[f'{rationale_name}_pred']==x['label']) ,axis=1)


df


Index(['source_id', 'content', 'label', 'publish_date', 'td_pred',
       'td_rationale'],
      dtype='object')
Index(['source_id', 'content', 'label', 'publish_date', 'td_pred',
       'td_rationale', 'td_acc', 'cs_pred', 'cs_rationale'],
      dtype='object')


,source_id,content,label,publish_date,td_pred,td_rationale,td_acc,cs_pred,cs_rationale,cs_acc
0,gossipcop-2493749932,Congratulations might be in order for Miley Cy...,1,1.529680e+09,1,1. **Source**: The source of the information ...,1,1,\n - Plausibility: While it is plausible tha...,1
1,gossipcop-941805037,Thousands are taking the streets to protest Pr...,1,1.492292e+09,0,\n1. **Specific Details**: The message includ...,0,0,Plausibility: The message discusses a real ev...,0
2,gossipcop-2547891536,We'd venture to say that Cindy Crawford's daug...,1,1.458333e+09,0,1. **Source and Context**: The message appear...,0,0,\n - Plausibility: The story is about Kaia G...,0
3,gossipcop-5189580095,"During the summer of 1995, John F. Kennedy Jr....",1,1.498769e+09,0,1. **Source**: The information is sourced fro...,0,0,\n - **Plausibility:** The story involves we...,0
4,gossipcop-9588339534,"Many stars saw great success in 2016, but the ...",1,1.481818e+09,0,\n1. **Specific Details**: The text provides ...,0,0,\n- Plausibility: The events described in the...,0
...,...,...,...,...,...,...,...,...,...,...
12326,gossipcop-907293,Gigi & Zayn Gave Us the Best Idea For a Valent...,0,1.485441e+09,1,- The message is written in a highly informal...,0,1,Plausibility: The article is filled with info...,0
12327,gossipcop-891749,2017 Hollywood Film Awards ceremony took place...,0,1.509974e+09,0,- The message provides specific details about...,1,0,Plausibility: The message describes a typical...,1
12328,gossipcop-941486,Jada Pinkett Smith recalled that heartbreaking...,0,1.528042e+09,0,"- The message is from a credible source, ""Red...",1,0,Plausibility: The story is plausible as it di...,1
12329,gossipcop-953143,Another one of Luann de Lesseps' Real Housewiv...,0,NaN,0,- The message discusses a specific and detail...,1,0,Plausibility: The message is about a Real Hou...,1


In [4]:
print(f"sum: {df.shape[0]}")
value_count = df['label'].value_counts()
print(f"real data: {value_count.get(1,0)}")
print(f"fake data: {value_count.get(0,0)}")

sum: 12331
real data: 2716
fake data: 9615


In [5]:
def filter_data(df):
    for r_name in rationale_names:
        df = df[ (df[f'{r_name}_pred'] != -1) & (df[f'{r_name}_rationale'] is not None) ]
    return df
        
print(f"before filtering : {df.shape[0]}  real data :{(df['label']==1).sum()} fake data: {(df['label']==0).sum()}")
df = filter_data(df)
print(f"after filtering : {df.shape[0]}  real data :{(df['label']==1).sum()} fake data: {(df['label']==0).sum()}")


before filtering : 12331  real data :2716 fake data: 9615
after filtering : 12331  real data :2716 fake data: 9615


In [7]:
from Util import cal_rationale_metrics
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score

for r_name in rationale_names:
    pred = df[r_name+ "_pred"].tolist()
    label = df['label'].tolist()
    print(cal_rationale_metrics(pred,label))
    

{'acc': 0.8342389100640661, 'recall': 0.6573948451310659, 'recall_real': 0.9734789391575663, 'recall_fake': 0.34131075110456555, 'precision': 0.8123455854121868, 'precision_real': 0.8397631437286919, 'precision_fake': 0.7849280270956817, 'f1_macro': 0.6887206236867336, 'f1_real': 0.9016906700062617, 'f1_fake': 0.47575057736720555}
{'acc': 0.8359419349606683, 'recall': 0.6609966401601572, 'recall_real': 0.9736869474778991, 'recall_fake': 0.3483063328424153, 'precision': 0.8160977632032494, 'precision_real': 0.841225626740947, 'precision_fake': 0.7909698996655519, 'f1_macro': 0.6931312634219706, 'f1_real': 0.9026224450443502, 'f1_fake': 0.483640081799591}


In [8]:
def split_data0(df,train_ratio=0.8,valid_ratio=0.1):
    train_nums = int(df.shape[0]*train_ratio)
    valid_nums = int(df.shape[0]*valid_ratio)
    return df.iloc[:train_nums],df.iloc[train_nums:train_nums+valid_nums],df.iloc[train_nums+valid_nums:]

def split_data(df,train_ratio=0.8,valid_ratio=0.1):
    real_data = df[df['label'] == 1]
    fake_data = df[df['label'] == 0]
    print(f"real_data count : {real_data.shape[0]}")
    print(f"fake_data count : {fake_data.shape[0]}")
    train_real,valid_real,test_real = split_data0(real_data,train_ratio,valid_ratio)
    train_fake,valid_fake,test_fake = split_data0(fake_data,train_ratio,valid_ratio)
    train_df,valid_df,test_df = pd.concat([train_real,train_fake]),pd.concat([valid_real,valid_fake]),pd.concat([test_real,test_fake])
    train_df['split'] = 'train'
    valid_df['split'] = 'val'
    test_df['split'] = 'test'
    return train_df,valid_df,test_df


train_df,valid_df,test_df = split_data(df,train_ratio=0.8,valid_ratio=0.1)


train_df.shape[0],valid_df.shape[0],test_df.shape[0]

real_data count : 2716
fake_data count : 9615


(9864, 1232, 1235)

In [9]:
train_df.to_csv('train.csv',index=False)
valid_df.to_csv('val.csv',index=False)
test_df.to_csv('test.csv',index=False)